In [50]:
import tensorflow as tf
import ltn
from ucimlrepo import fetch_ucirepo
from sklearn.model_selection import train_test_split
import commons
from collections import defaultdict

In [37]:
poker = fetch_ucirepo(id=158)
X, y = poker.data.features, poker.data.targets

X_train, X_temp, y_train, y_temp = train_test_split(
        X, y, test_size=0.2, stratify=y, random_state=42)

X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)

X_np = X_train.to_numpy()
y_np = y_train.to_numpy()
ds_train = tf.data.Dataset.from_tensor_slices((X_np, y_np))
X_np = X_val.to_numpy()
y_np = y_val.to_numpy()
ds_val = tf.data.Dataset.from_tensor_slices((X_np, y_np))
X_np = X_test.to_numpy()
y_np = y_test.to_numpy()
ds_test = tf.data.Dataset.from_tensor_slices((X_np, y_np))

In [38]:
class Model(tf.keras.Model):
    def __init__(self, n_hidden=32, n_classes=10):
        super().__init__()
        self.dense1 = tf.keras.layers.Dense(n_hidden, activation="relu")
        self.dense2 = tf.keras.layers.Dense(n_classes, activation="softmax")

    def call(self, inputs):
        hand,label = inputs
        x = self.dense1(hand)
        probs = self.dense2(x)
        truth = tf.reduce_sum(probs * label, axis=-1, keepdims=True)
        return truth

In [39]:
# Predicates
HandType = ltn.Predicate(Model())
# Operators
Not     = ltn.Wrapper_Connective(ltn.fuzzy_ops.Not_Std())
And     = ltn.Wrapper_Connective(ltn.fuzzy_ops.And_Prod())
Or      = ltn.Wrapper_Connective(ltn.fuzzy_ops.Or_ProbSum())
Implies = ltn.Wrapper_Connective(ltn.fuzzy_ops.Implies_Reichenbach())

Forall  = ltn.Wrapper_Quantifier(ltn.fuzzy_ops.Aggreg_pMeanError(p=4), 'forall')  # ∀
Exists  = ltn.Wrapper_Quantifier(ltn.fuzzy_ops.Aggreg_pMean(),        'exists')   # ∃
Equiv   = ltn.Wrapper_Connective(ltn.fuzzy_ops.Equiv(And, Implies))


In [40]:
def bincount_batch(ranks, length=14): #13 bins 1-13
    return tf.map_fn( #apply to every row
        #count how many times each value appears in the input
        #and cast the output to float 32 to be used by predicates and fuzzy operations
        lambda row: tf.cast(tf.math.bincount(row,
                                             minlength=length,
                                             maxlength=length), tf.float32),
        ranks)

def same_suit(hand):
    suits  = hand[:, 0:10:2]
    truth  = tf.reduce_all(suits[:, 1:] == suits[:, :1], axis=1, keepdims=True)
    return tf.cast(truth, tf.float32)

def two_of_a_kind(hand):
    counts   = bincount_batch(hand[:, 1:10:2])            # [b,14]
    twos     = tf.reduce_sum(tf.cast(counts == 2, tf.float32), axis=1, keepdims=True)
    threes   = tf.reduce_sum(tf.cast(counts == 3, tf.float32), axis=1, keepdims=True)
    fours    = tf.reduce_sum(tf.cast(counts == 4, tf.float32), axis=1, keepdims=True)
    truth    = tf.cast((twos == 1) & (threes == 0) & (fours == 0), tf.float32)
    return truth

def two_of_two_kinds(hand):
    counts = bincount_batch(hand[:, 1:10:2])
    pairs  = tf.reduce_sum(tf.cast(counts == 2, tf.float32), axis=1, keepdims=True)
    truth  = tf.cast(pairs == 2, tf.float32)
    return truth

def three_of_a_kind(hand):
    counts  = bincount_batch(hand[:, 1:10:2])
    threes  = tf.reduce_sum(tf.cast(counts == 3, tf.float32), axis=1, keepdims=True)
    twos    = tf.reduce_sum(tf.cast(counts == 2, tf.float32), axis=1, keepdims=True)
    truth   = tf.cast((threes == 1) & (twos == 0), tf.float32)
    return truth

def four_of_a_kind(hand):
    counts = bincount_batch(hand[:, 1:10:2])
    quads  = tf.reduce_sum(tf.cast(counts == 4, tf.float32), axis=1, keepdims=True)
    return tf.cast(quads == 1, tf.float32)

def two_of_a_kind_three_of_a_kind(hand):       # full house
    counts  = bincount_batch(hand[:, 1:10:2])
    threes  = tf.reduce_sum(tf.cast(counts == 3, tf.float32), axis=1, keepdims=True)
    twos    = tf.reduce_sum(tf.cast(counts == 2, tf.float32), axis=1, keepdims=True)
    return tf.cast((threes == 1) & (twos == 1), tf.float32)

def five_sequence(hand):                       # straight
    ranks, _  = tf.sort(hand[:, 1:10:2], axis=1)
    diffs     = ranks[:, 1:] - ranks[:, :-1]
    truth     = tf.reduce_all(diffs == 1, axis=1, keepdims=True)
    return tf.cast(truth, tf.float32)

def straight_flush(hand):
    return tf.cast((same_suit(hand) == 1.0) & (five_sequence(hand) == 1.0), tf.float32)

def royal_flush(hand):
    royal   = tf.constant([1,10,11,12,13], dtype=tf.int32)
    ranks   = tf.sort(hand[:, 1:10:2], axis=1)[0]
    match   = tf.reduce_all(tf.equal(ranks, royal), axis=1, keepdims=True)
    return tf.cast((same_suit(hand) == 1.0) & match, tf.float32)

def nothing_hand(hand):
    truth = ~(
        (two_of_a_kind(hand)               == 1.0) |
        (two_of_two_kinds(hand)            == 1.0) |
        (three_of_a_kind(hand)             == 1.0) |
        (five_sequence(hand)               == 1.0) |
        (same_suit(hand)                   == 1.0) |
        (two_of_a_kind_three_of_a_kind(hand) == 1.0) |
        (four_of_a_kind(hand)              == 1.0) |
        (straight_flush(hand)              == 1.0) |
        (royal_flush(hand)                 == 1.0)
    )
    return tf.cast(truth, tf.float32)

equals = ltn.Predicate.Lambda(
    lambda x, y: tf.cast(
        tf.reduce_all(tf.equal(x, y), axis=-1, keepdims=True),
        tf.float32)
)
NothingRule       = ltn.Predicate.Lambda(nothing_hand)
OnePairRule       = ltn.Predicate.Lambda(two_of_a_kind)
TwoPairRule       = ltn.Predicate.Lambda(two_of_two_kinds)
ThreeOfAKindRule  = ltn.Predicate.Lambda(three_of_a_kind)
StraightRule      = ltn.Predicate.Lambda(five_sequence)
FlushRule         = ltn.Predicate.Lambda(same_suit)
FullHouseRule     = ltn.Predicate.Lambda(two_of_a_kind_three_of_a_kind)
FourOfAKindRule   = ltn.Predicate.Lambda(four_of_a_kind)
StraightFlushRule = ltn.Predicate.Lambda(straight_flush)
RoyalFlushRule    = ltn.Predicate.Lambda(royal_flush)

In [41]:
Nothing, OnePair, TwoPairs, ThreeOfAKind,Straight,Flush,FullHouse,FourOfAKind, StraightFlush, RoyalFlush = tf.eye(10)
@tf.function
def axioms(hand,label):
    # Variables
    hand_variable = ltn.Variable("hand_variable", hand)
    label_variable = ltn.Variable("label_variable", label)
    #this creates a tensor for every class
    
    
    label_domain = tf.eye(10)
    l1 = ltn.Variable("l1",label_domain)
    l2 = ltn.Variable("l2",label_domain)
    supervised_rule = Forall(ltn.diag(hand_variable,label_variable), HandType([hand_variable, label_variable]), p=2)
    
    # x_rule(hand) -> hand has label x
    rule_axioms = [
        Forall(hand_variable,
               Implies(
                   NothingRule(hand_variable),
                   HandType([hand_variable,Nothing])), p=2),
        Forall(hand_variable,
               Implies(
                   OnePairRule(hand_variable),
                   HandType([hand_variable,OnePair])), p=2),
        Forall(hand_variable,
               Implies(
                   TwoPairRule(hand_variable),
                   HandType([hand_variable,TwoPairs])), p=2),
        Forall(hand_variable,
               Implies(
                   ThreeOfAKindRule(hand_variable),
                   HandType([hand_variable,ThreeOfAKind])), p=2),
        Forall(hand_variable,
               Implies(
                   StraightRule(hand_variable),
                   HandType([hand_variable,Straight])), p=2),
        Forall(hand_variable,
               Implies(
                   FlushRule(hand_variable),
                   HandType([hand_variable,Flush])), p=2),
        Forall(hand_variable,
               Implies(
                   FullHouseRule(hand_variable),
                   HandType([hand_variable,FullHouse])), p=2),
        Forall(hand_variable,
               Implies(
                   FourOfAKindRule(hand_variable),
                   HandType([hand_variable,FourOfAKind])), p=2),
        Forall(hand_variable,
               Implies(
                   StraightFlushRule(hand_variable),
                   HandType([hand_variable,StraightFlush])), p=2),
        Forall(hand_variable,
               Implies(
                   RoyalFlushRule(hand_variable),
                   HandType([hand_variable,RoyalFlush])), p=2),
    ]
    #a hand cannot have two types at once
    exclusivity_rule = Implies(~equals(l1, l2),
            ~(And(HandType([hand_variable, l1]),
                      HandType([hand_variable, l2]))))
    
    all_axioms = ltn.LTNCollection([supervised_rule, *rule_axioms, exclusivity_rule])

    return all_axioms.tensor

x1, y1 = next(ds_train.as_numpy_iterator())
axioms(x1,y1)

OperatorNotAllowedInGraphError: in user code:

    File "C:\Users\andre\AppData\Local\Temp\ipykernel_20184\1119039408.py", line 16, in axioms  *
        rule_axioms = [
    File "C:\Users\andre\Desktop\RP\.venv312\Lib\site-packages\ltn\core.py", line 189, in __call__  *
        expr = super().__call__(inputs, *args, **kwargs)
    File "C:\Users\andre\Desktop\RP\.venv312\Lib\site-packages\ltn\core.py", line 155, in __call__  *
        t_outputs = self.model(flat_inputs[0].tensor, *args, **kwargs)
    File "C:\Users\andre\Desktop\RP\.venv312\Lib\site-packages\keras\src\utils\traceback_utils.py", line 122, in error_handler  **
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\andre\Desktop\RP\.venv312\Lib\site-packages\ltn\core.py", line 306, in call
        return self.lambda_layer(inputs)
    File "C:\Users\andre\AppData\Local\Temp\ipykernel_20184\3742157694.py", line 67, in nothing_hand
        (five_sequence(hand)               == 1.0) |
    File "C:\Users\andre

In [43]:
Nothing, OnePair, TwoPairs, ThreeOfAKind,Straight,Flush,FullHouse,FourOfAKind, StraightFlush, RoyalFlush = tf.eye(10)
@tf.function
def axioms(hand,label,p_schedule):
    hand_variable = ltn.Variable("hand_variable", hand)
    label_variable = ltn.Variable("label_variable", label)
    #this creates a tensor for every class
    
    
    label_domain = tf.eye(10)
    l1 = ltn.Variable("l1",label_domain)
    l2 = ltn.Variable("l2",label_domain)
    supervised_rule = Forall(ltn.diag(hand_variable,label_variable), HandType([hand_variable, label_variable]), p=2)
    
    # x_rule(hand) -> hand has label x
    rule_axioms = [
        Forall(hand_variable,
               Implies(
                   NothingRule(hand_variable),
                   HandType([hand_variable,Nothing])), p=2),
        Forall(hand_variable,
               Implies(
                   OnePairRule(hand_variable),
                   HandType([hand_variable,OnePair])), p=2),
        Forall(hand_variable,
               Implies(
                   TwoPairRule(hand_variable),
                   HandType([hand_variable,TwoPairs])), p=2),
        Forall(hand_variable,
               Implies(
                   ThreeOfAKindRule(hand_variable),
                   HandType([hand_variable,ThreeOfAKind])), p=2),
        Forall(hand_variable,
               Implies(
                   StraightRule(hand_variable),
                   HandType([hand_variable,Straight])), p=2),
        Forall(hand_variable,
               Implies(
                   FlushRule(hand_variable),
                   HandType([hand_variable,Flush])), p=2),
        Forall(hand_variable,
               Implies(
                   FullHouseRule(hand_variable),
                   HandType([hand_variable,FullHouse])), p=2),
        Forall(hand_variable,
               Implies(
                   FourOfAKindRule(hand_variable),
                   HandType([hand_variable,FourOfAKind])), p=2),
        Forall(hand_variable,
               Implies(
                   StraightFlushRule(hand_variable),
                   HandType([hand_variable,StraightFlush])), p=2),
        Forall(hand_variable,
               Implies(
                   RoyalFlushRule(hand_variable),
                   HandType([hand_variable,RoyalFlush])), p=2),
    ]
    #a hand cannot have two types at once
    exclusivity_rule = Implies(~equals(l1, l2),
            ~(And(HandType([hand_variable, l1]),
                      HandType([hand_variable, l2]))))
    
    all_axioms = ltn.LTNCollection([supervised_rule, *rule_axioms, exclusivity_rule])

    return all_axioms.tensor

x1, y1 = next(ds_train.as_numpy_iterator())
axioms(x1,y1, tf.constant(2.))

OperatorNotAllowedInGraphError: in user code:

    File "C:\Users\andre\AppData\Local\Temp\ipykernel_20184\186501409.py", line 15, in axioms  *
        rule_axioms = [
    File "C:\Users\andre\Desktop\RP\.venv312\Lib\site-packages\ltn\core.py", line 189, in __call__  *
        expr = super().__call__(inputs, *args, **kwargs)
    File "C:\Users\andre\Desktop\RP\.venv312\Lib\site-packages\ltn\core.py", line 155, in __call__  *
        t_outputs = self.model(flat_inputs[0].tensor, *args, **kwargs)
    File "C:\Users\andre\Desktop\RP\.venv312\Lib\site-packages\keras\src\utils\traceback_utils.py", line 122, in error_handler  **
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\andre\Desktop\RP\.venv312\Lib\site-packages\ltn\core.py", line 306, in call
        return self.lambda_layer(inputs)
    File "C:\Users\andre\AppData\Local\Temp\ipykernel_20184\3742157694.py", line 67, in nothing_hand
        (five_sequence(hand)               == 1.0) |
    File "C:\Users\andre\

In [44]:
optimizer = tf.keras.optimizers.Adam(0.001)
metrics_dict = {
    'train_loss': tf.keras.metrics.Mean(name="train_loss"),
    'train_accuracy': tf.keras.metrics.Mean(name="train_accuracy"),
    'test_loss': tf.keras.metrics.Mean(name="test_loss"),
    'test_accuracy': tf.keras.metrics.Mean(name="test_accuracy")    
}

In [45]:
@tf.function
def train_step(hand,label, **parameters):
    # loss
    with tf.GradientTape() as tape:
        loss = 1.- axioms(hand, label, **parameters)
    gradients = tape.gradient(loss, HandType.trainable_variables)
    optimizer.apply_gradients(zip(gradients, HandType.trainable_variables))
    metrics_dict['train_loss'](loss)
    # accuracy
    predictions = tf.argmax(HandType([hand]),axis=-1)
    match = tf.equal(label,tf.cast(predictions,label.dtype))
    metrics_dict['train_accuracy'](tf.reduce_mean(tf.cast(match,tf.float32)))

In [46]:
@tf.function
def test_step(hand,label, **parameters):
    # loss
    loss = 1.- axioms(hand,label, **parameters)
    metrics_dict['test_loss'](loss)
    # accuracy
    predictions = tf.argmax(HandType([hand]),axis=-1)
    match = tf.equal(label,tf.cast(predictions,label.dtype))
    metrics_dict['train_accuracy'](tf.reduce_mean(tf.cast(match,tf.float32)))

In [51]:
scheduled_parameters = defaultdict(lambda: {})
for epoch in range(0,4):
    scheduled_parameters[epoch] = {"p_schedule":tf.constant(1.)}
for epoch in range(4,8):
    scheduled_parameters[epoch] = {"p_schedule":tf.constant(2.)}
for epoch in range(8,12):
    scheduled_parameters[epoch] = {"p_schedule":tf.constant(4.)}
for epoch in range(12,20):
    scheduled_parameters[epoch] = {"p_schedule":tf.constant(6.)}
commons.train(
    20,
    metrics_dict,
    ds_train,
    ds_test,
    train_step,
    test_step,
    scheduled_parameters=scheduled_parameters
)

OperatorNotAllowedInGraphError: in user code:

    File "C:\Users\andre\AppData\Local\Temp\ipykernel_20184\592393324.py", line 5, in train_step  *
        loss = 1.- axioms(hand, label, **parameters)
    File "C:\Users\andre\AppData\Local\Temp\ipykernel_20184\186501409.py", line 15, in axioms  *
        rule_axioms = [
    File "C:\Users\andre\Desktop\RP\.venv312\Lib\site-packages\ltn\core.py", line 189, in __call__  *
        expr = super().__call__(inputs, *args, **kwargs)
    File "C:\Users\andre\Desktop\RP\.venv312\Lib\site-packages\ltn\core.py", line 155, in __call__  *
        t_outputs = self.model(flat_inputs[0].tensor, *args, **kwargs)
    File "C:\Users\andre\Desktop\RP\.venv312\Lib\site-packages\keras\src\utils\traceback_utils.py", line 122, in error_handler  **
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\andre\Desktop\RP\.venv312\Lib\site-packages\ltn\core.py", line 306, in call
        return self.lambda_layer(inputs)
    File "C:\Users\andre\A